### Trabalho de Algoritmos genéticos - Construção de dietas usando a inteligência artificial

Algoritmo base => Mochila

In [1]:
#Importações
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import random

from funcoes import computa_cesta
from funcoes import funcao_objetivo_pop_dieta
from funcoes import populacao_cesta_alimento as cria_populacao_inicial
from funcoes import selecao_roleta_max as funcao_selecao
from funcoes import cruzamento_ponto_simples as funcao_cruzamento
from funcoes import mutacao_dieta as funcao_mutacao

#### Obtendo informações do usuário:

A primeira parte do código consiste em coletar informações do usuário que fará o uso do nosso programa

In [2]:
dados = []
def informacoes_da_pessoa():

    advertencia = ["Digite uma resposta válida"]
    

    print("Qual é o seu nome?")
    nome = input()
    dados.append(nome)

    print("Qual a sua idade?")
    idade_str = input()
    idade = float(idade_str)
    if idade in range(18,120):
        dados.append(idade) 
    else:
        print(advertencia)
        idade = input()

    print("Qual o seu peso?")
    peso_str = input()
    peso = float(peso_str)
    dados.append(peso)

    print("Qual é o seu sexo?")
    sexo = input()
    while sexo.lower() not in ["feminino","masculino"]:
        print(advertencia)
        sexo = input()
    dados.append(sexo)

    print("Qual é a sua altura?")
    altura_str = input()
    altura = float(altura_str)
    dados.append(altura)

    print("Qual é o seu nível de atividade física?\n sedentário: pouco ou nenhum exercício físico\n pouco ativo:leve 1-3/semana\n moderadamente ativo: moderado 3-5/semana\n muito ativo: pesado 7-7 dias/semana\n extremamente ativo: pesado/intenso ou treino 2xdia")
    atividade = input()
    while atividade.lower() not in ["sedentário", "pouco ativo", "moderadamente ativo", "muito ativo", "extremamente ativo"]:
        print(advertencia)
        atividade = input()
    dados.append(atividade)
    
    print("Qual é o seu objetivo com a dieta?\n manter massa\n perder massa\n ganhar massa")
    objetivo = input()
    while objetivo.lower() not in ["manter massa", "perder massa", "ganhar massa"]:
        print(advertencia)
        objetivo = input()
    dados.append(objetivo)
    
    return dados

print(informacoes_da_pessoa())

Qual é o seu nome?


Qual a sua idade?


ValueError: could not convert string to float: ''

#### Como determinaremos a quantidade de calorias ideais?

A equação de Harris-Benedict é uma fórmula que utiliza a altura, peso, idade e sexo de determinado indivíduo para calcular a sua taxa metabólica basal(TMB)

**1º Passo:**
Determinar a taxa metabólica basal (TMB)

Homens: TMB = 66 + (13.7 x peso em quilogramas) + (5 x altura em centímetros) – (6.8 x idade em anos)

Mulheres: TMB = 65.5 + (9.6 X peso em quilogramas) + (1.8 X altura em centímetros) – (4.7 x idade em anos).

**2º Passo:**
Multiplicar o valor da TMB pelo fator de atividade física adequado

- Sedentário – pouco ou nenhum exercício (TMB x 1.2)

- Pouco ativo – exercício/esporte leve 1-3 dias/semana (TMB x 1.375)

- Moderadamente ativo – exercício/esporte moderado 3-5 dias/semana (TMB x 1.55)

- Muito ativo – exercício/esporte pesado 6-7 dias/semana (TMB x 1.725)

- Extremamente ativo – exercício/esporte muito pesado e trabalho físico intenso diariamente ou treino de 2x ao dia (TMB x 1.9)

Este cálculo vai te dar um valor estimado das necessidades energéticas diárias de uma pessoa. Contudo, não se esqueça que se trata apenas de uma estimativa e que deve ser usada como um guia e não como uma regra.

In [3]:
altura = dados[4]
peso = dados[2]
idade = dados[1]
sexo = dados[3]
atividade = dados[5]
objetivo = dados[6]


IndexError: list index out of range

In [4]:
def tmb_func(altura, peso, idade, sexo, atividade,objetivo):
    if sexo == "feminino":
        tmb_1 = 655.1 + (9.6 * peso) + (1.8 * altura) - (4.7 * idade)
    else:
        tmb_1= 66.5 + (13.7 * peso) + (5 * altura) - (6.8 * idade)
        
    if atividade == "sedentário":
        tmb = tmb_1 * 1.2
    elif atividade == "pouco ativo":
        tmb = tmb_1 * 1.375
    elif atividade == "moderadamente ativo":
        tmb = tmb_1 * 1.55
    elif atividade == "muito ativo":
        tmb = tmb_1 * 1.725
    else:
        tmb = tmb_1 * 1.9
        
    if objetivo.lower() == "manter massa":
        
        tmb_final = tmb
        
    elif objetivo.lower() == "perder massa":
        
        tmb_final = tmb - 500
        
    else:
        
        tmb_final = tmb + 500
        
    return tmb_final

print(tmb_func(altura,peso,idade,sexo, atividade, objetivo))

NameError: name 'altura' is not defined

#### Agora que temos todas as informações necessárias, vamos enfim começar a criar o nosso código do nutrella

In [5]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 20
NUM_GERACOES = 100
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.1

# relacionadas ao problema a ser resolvido
# LIMITE_DE_CALORIAS = tmb_func(altura,peso,idade,sexo, atividade, objetivo)
LIMITE_DE_CALORIAS = 2000


# O supermercado

# Leia o arquivo Excel como um DataFrame
df_alimentos = pd.read_excel("Alimentos.xlsx")

#Agora precisamos transformar esses df em dicionários(com orientação por linhas);
dic_alimentos = df_alimentos.to_dict(orient='records')

df_alimentosC = pd.read_excel("Alimentos - Calorias.xlsx")
dic_alimentosC = df_alimentosC.to_dict(orient='records')

SUPERMERCADO = {}
for item in dic_alimentosC:
    alimento = item['Alimentos']
    calorias_dic = item['Energia(Kcal)']
    Densidade_nutricional = item['Densidade nutricional']
    SUPERMERCADO[alimento] = {'calorias': calorias_dic, 'Densidade nutricional': Densidade_nutricional}
    
NUM_ALIMENTOS = len(SUPERMERCADO)
ORDEM_DOS_NOMES = list(sorted(SUPERMERCADO.keys()))

In [6]:
# Funções locais
print(LIMITE_DE_CALORIAS)
def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_dieta(populacao, SUPERMERCADO, LIMITE_DE_CALORIAS, ORDEM_DOS_NOMES)

2000


In [7]:
# Busca por algoritmo genético


populacao = cria_populacao_inicial(TAMANHO_POP, NUM_ALIMENTOS)

 

# variaveis para o hall da fama
melhor_fitness_ja_visto = -float("inf") # levando em consideração que é um problema de maximização
melhor_individuo_ja_visto = [0] * NUM_ALIMENTOS # solução trivial para o problema

 

for n in range(NUM_GERACOES):
 

    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)
 

    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]

 

    contador = 0

 

    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2

 

        contador = contador + 2

 

    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)

 

    # melhor individuo já visto até agora (hall da fama)
    fitness = funcao_objetivo_pop(populacao)
    maior_fitness = max(fitness)
    posicao = fitness.index(maior_fitness)
    individuo = populacao[posicao].copy()
    densidade_nutri_alimento, calorias = computa_cesta(individuo, SUPERMERCADO, ORDEM_DOS_NOMES)
    if maior_fitness > melhor_fitness_ja_visto : # não mostrar coisas q n resolvem o problema
        melhor_fitness_ja_visto = maior_fitness
        melhor_individuo_ja_visto = individuo
        print(f"Maior densidade nutricional: {densidade_nutri_alimento} | Calorias: {calorias}")

#Resultado
        
print("Você deve pegar os seguintes itens:")
for pega_ou_nao, item in zip(melhor_individuo_ja_visto, ORDEM_DOS_NOMES):
    if pega_ou_nao == 1:
        print("+", item)
print()
valor_total, peso_total = computa_cesta(
    melhor_individuo_ja_visto, SUPERMERCADO, ORDEM_DOS_NOMES
)
print(
    f"Com isso, sua mochila terá o valor nutricional de {valor_total}"
    f"e quantidade calóricade de {peso_total} kcal."
)

Maior densidade nutricional: 135.0777107499845 | Calorias: 1672.1499999999999
Maior densidade nutricional: 176.7069638597586 | Calorias: 1626.99
Maior densidade nutricional: 180.4161688806791 | Calorias: 1865.9899999999998
Maior densidade nutricional: 188.53301222227452 | Calorias: 1826.36
Maior densidade nutricional: 201.50622650798883 | Calorias: 1854.36
Você deve pegar os seguintes itens:
+ Atum em conserva
+ Banana (ouro, prata, d´água, da terra, etc.)
+ Bebida achocolatada
+ Caqui
+ Cenoura amarela (batata-baroa)
+ Cogumelo in natura
+ Couve-flor
+ Geleia de frutas de qualquer marca ou sabor
+ Leite condensado
+ Mandioquinha salsa (batata-baroa)
+ Maracujá
+ Peito de galinha ou frango
+ Pitanga
+ Soja em grão
+ Tangerina
+ Tomate
+ Vagem

Com isso, sua mochila terá o valor nutricional de 201.50622650798883e quantidade calóricade de 1854.36 kcal.


### Referências:

[1] https://www.omnicalculator.com/health/bmr-harris-benedict-equation

[2] https://en.wikipedia.org/wiki/Harris%E2%80%93Benedict_equation

[3] https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5074671/

[4] https://www.sanarmed.com/equacao-de-harris-benedict-colunistas